In [16]:
import openai, json

client = openai.OpenAI()
messages = []

In [17]:
def get_weather(city):
  return "33 degrees celcius"

FUNCTION_MAP = {
  'get_weather': get_weather
}

In [18]:
TOOLS = [
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "A funtion to get the weather of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "city": {
            "type": "string",
            "description": "The name of the city to get the weather of."
          }
        },
        "required": ["city"]
      }
    }
  }
]

In [21]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
  
  if message.tool_calls:
    messages.append({
      "role": "assistant",
      "content": message.content or "",
      "tool_calls": [        
        {
          "id": tool_call.id,
          "type": "function",
          "function" : {
            "name": tool_call.function.name,
            "arguments": tool_call.function.arguments
          }
        } for tool_call in message.tool_calls],
    })

    for tool_call in message.tool_calls:
      function_name = tool_call.function.name
      arguments = tool_call.function.arguments

      print(f"Calling funtion: {function_name} with {arguments}")

      try:
        arguments = json.loads(arguments)
      except json.JSONDecodeError:
        arguments = {}
      
      funtion_to_run = FUNCTION_MAP.get(function_name)

      result = funtion_to_run(**arguments)

      print(f"Run {function_name} with args {arguments} for a result of {result}")

      messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": function_name,
        "content": result
      })
    
    call_ai()
  else :
    messages.append({"role": "assistant", "content": message.content})
    print(f"AI: {message.content}")
    
def call_ai():
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      tools=TOOLS
    )
  process_ai_response(response.choices[0].message)

In [22]:
while True:
  message = input("Send a message to the LLM...")
  if message == "quit" or message == "q":
    break
  else:
    messages.append({"role": "user", "content": message})
    print(f"User: {message}")
    call_ai()
   

User: My name is Nico
AI: Hello Nico! How can I assist you today?
User: What is my name
AI: Your name is Nico.
User: What is the weather in Korea
Calling funtion: get_weather with {"city":"Korea"}
Run get_weather with args {'city': 'Korea'} for a result of 33 degrees celcius
AI: The weather in Korea is currently 33 degrees Celsius. Would you like to know anything else?
User: 한국말로 번역해줘
AI: 한국의 날씨는 현재 33도 섭씨입니다. 다른 궁금한 점이 있으신가요?
